In [1]:
import pandas as pd
import numpy as np

# json begins

### This is just a warm-up exercise. The "JSON exercise" comment below is followed by the actual assessment.

In [2]:
import json
from pandas.io.json import json_normalize

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,shortname,info.governor,state
0,Dade,12345,FL,Rick Scott,Florida
1,Broward,40000,FL,Rick Scott,Florida
2,Palm Beach,60000,FL,Rick Scott,Florida
3,Summit,1234,OH,John Kasich,Ohio
4,Cuyahoga,1337,OH,John Kasich,Ohio


## JSON exercise 
### Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above, 
#### Find the 10 countries with most projects 
#### Find the top 10 major project themes (using column 'mjtheme_namecode') 
#### In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [6]:
wb_json = pd.read_json('./data/data_wrangling_json/data/world_bank_projects.json')

In [7]:
wb_json.head(2)

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


In [8]:
len(wb_json.index)

500

In [9]:
#Create a new function:
def num_missing(x):
    return sum(x.isnull())

In [10]:
#Applying per column
print "Missing values per column"
print wb_json.apply(num_missing, axis=0).head() #axis=0 -> apply column-wise / for each column

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-10-747b65020c81>, line 2)

In [122]:
#non missing in "id" - use this to pivot  
#Create pivot table
grps = wb_json.pivot_table(values=["_id"], index=["countryshortname"], aggfunc=len)
print (grps.head())

                     _id
countryshortname        
Afghanistan            6
Africa                11
Albania                4
Angola                 1
Antigua and Barbuda    1


## 1. Top 10 Countries with most projects

In [124]:
grps.sort_values("_id", ascending=False).head(n=10)

,_id
countryshortname,
China,19
Indonesia,19
Vietnam,17
India,16
"Yemen, Republic of",13
Nepal,12
Bangladesh,12
Morocco,12
Mozambique,11


## next exercise (need help here) ... 2. Top 10 project themes
###### approach - determine the unique themes and their frequencies & slice the top 10 to get the solution

###### exploring the data and types...

In [17]:
print (wb_json['mjtheme_namecode'].head())

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object


In [18]:
print (wb_json['mjtheme_namecode'][0])
#wb_json['mjtheme_namecode']

[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]


In [19]:
print (wb_json[['themecode', 'mjtheme_namecode']][:5])

     themecode                                   mjtheme_namecode
0           65  [{'code': '8', 'name': 'Human development'}, {...
1        54,24  [{'code': '1', 'name': 'Economic management'},...
2  52,81,25,47  [{'code': '5', 'name': 'Trade and integration'...
3        59,57  [{'code': '7', 'name': 'Social dev/gender/incl...
4        41,45  [{'code': '5', 'name': 'Trade and integration'...


In [21]:
print (type(wb_json['mjtheme_namecode']))
print (type(wb_json['mjtheme_namecode'][0]))
print (type(wb_json['mjtheme_namecode'][0][0]))
###woooo a series of lists of dictionaries... enjoy!! :D

<class 'pandas.core.series.Series'>
<class 'list'>
<class 'dict'>


In [22]:
print (wb_json['mjtheme_namecode'][:5])

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object


#### this will help separate out individual themes

In [24]:
#unraveling the column 
for x in wb_json['mjtheme_namecode']:
    print (x)
    for y in x:
        print (y)

[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]
{'code': '8', 'name': 'Human development'}
{'code': '11', 'name': ''}
[{'code': '1', 'name': 'Economic management'}, {'code': '6', 'name': 'Social protection and risk management'}]
{'code': '1', 'name': 'Economic management'}
{'code': '6', 'name': 'Social protection and risk management'}
[{'code': '5', 'name': 'Trade and integration'}, {'code': '2', 'name': 'Public sector governance'}, {'code': '11', 'name': 'Environment and natural resources management'}, {'code': '6', 'name': 'Social protection and risk management'}]
{'code': '5', 'name': 'Trade and integration'}
{'code': '2', 'name': 'Public sector governance'}
{'code': '11', 'name': 'Environment and natural resources management'}
{'code': '6', 'name': 'Social protection and risk management'}
[{'code': '7', 'name': 'Social dev/gender/inclusion'}, {'code': '7', 'name': 'Social dev/gender/inclusion'}]
{'code': '7', 'name': 'Social dev/gender/inclusion'}
{'code': 

In [106]:
s = wb_json['mjtheme_namecode']
#print(s)

In [113]:
cou=0
for l in s:
    for d in l:
        df.loc[cou]= d
        cou +=1

In [115]:
df.head()

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


In [121]:
gs = df.pivot_table(values=["name"], index=["code"], aggfunc=len)
print (gs.head())

      name
code      
1       38
10     216
11     250
2      199
3       15


In [26]:
for x in s:
    for y in x:
        #print (y)
        

{'code': '8', 'name': 'Human development'}
{'code': '11', 'name': ''}
{'code': '1', 'name': 'Economic management'}
{'code': '6', 'name': 'Social protection and risk management'}
{'code': '5', 'name': 'Trade and integration'}
{'code': '2', 'name': 'Public sector governance'}
{'code': '11', 'name': 'Environment and natural resources management'}
{'code': '6', 'name': 'Social protection and risk management'}
{'code': '7', 'name': 'Social dev/gender/inclusion'}
{'code': '7', 'name': 'Social dev/gender/inclusion'}
{'code': '5', 'name': 'Trade and integration'}
{'code': '4', 'name': 'Financial and private sector development'}
{'code': '6', 'name': 'Social protection and risk management'}
{'code': '6', 'name': ''}
{'code': '2', 'name': 'Public sector governance'}
{'code': '4', 'name': 'Financial and private sector development'}
{'code': '11', 'name': 'Environment and natural resources management'}
{'code': '8', 'name': ''}
{'code': '10', 'name': 'Rural development'}
{'code': '7', 'name': ''}


#### now collect the individual themes and their frequencies in a dictionary called counter
note: dlist is temporary

In [16]:
#collecting distinct elements and their frequencies
dlist = []
counter = {}
cou = 0
for x in wb_json['mjtheme_namecode']:
    for y in x:
        if y in dlist:
            for z in counter.values():
                if z['code'] == y.values()[0]:
                    z['freq'] += 1 
        else:
            dlist.append(y.copy())
            counter[cou] = {'code': y.values()[0], 'freq':1, 'name':y.values()[1]}
            cou += 1

In [174]:
#print dlist
print counter

{0: {'freq': 209, 'code': u'8', 'name': u'Human development'}, 1: {'freq': 249, 'code': u'11', 'name': u''}, 2: {'freq': 37, 'code': u'1', 'name': u'Economic management'}, 3: {'freq': 167, 'code': u'6', 'name': u'Social protection and risk management'}, 4: {'freq': 76, 'code': u'5', 'name': u'Trade and integration'}, 5: {'freq': 198, 'code': u'2', 'name': u'Public sector governance'}, 6: {'freq': 249, 'code': u'11', 'name': u'Environment and natural resources management'}, 7: {'freq': 129, 'code': u'7', 'name': u'Social dev/gender/inclusion'}, 8: {'freq': 145, 'code': u'4', 'name': u'Financial and private sector development'}, 9: {'freq': 165, 'code': u'6', 'name': u''}, 10: {'freq': 209, 'code': u'8', 'name': u''}, 11: {'freq': 215, 'code': u'10', 'name': u'Rural development'}, 12: {'freq': 128, 'code': u'7', 'name': u''}, 13: {'freq': 194, 'code': u'2', 'name': u''}, 14: {'freq': 144, 'code': u'4', 'name': u''}, 15: {'freq': 49, 'code': u'9', 'name': u'Urban development'}, 16: {'freq

#### at this point, counter has the list of themes with their respective frequencies. Need to get the top 10 themes for exercise #2 and fill in the empty "names" for themes based on their codes for exercise #3.

In [20]:
#print counter[0]['code']
cou = 0
for i in counter.values():
    #print i['code']
    if i['code'] == '8':
        print "found it"
cou += 1
#print cou

found it
found it


In [226]:
#collecting distinct elements and their frequencies
df = pd.DataFrame(wb_json['mjtheme_namecode'][0])
for i in wb_json['mjtheme_namecode'].index:
    df.append(wb_json['mjtheme_namecode'][i])

In [224]:
df
print i

499


In [225]:
wb_json['mjtheme_namecode'][i]

[{u'code': u'9', u'name': u'Urban development'},
 {u'code': u'8', u'name': u'Human development'},
 {u'code': u'5', u'name': u'Trade and integration'},
 {u'code': u'4', u'name': u'Financial and private sector development'}]